In [1]:
# Importation des librairies
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate

In [2]:
heart_df = pd.read_csv('../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')
heart_df['HadHeartAttack'] = heart_df['HadHeartAttack'].replace({'Yes': True, 'No': False}).astype(bool)
heart_df

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_16055/1405521179.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  heart_df['HadHeartAttack'] = heart_df['HadHeartAttack'].replace({'Yes': True, 'No': False}).astype(bool)


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,False,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,False,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",False,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,False,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,False,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,Virgin Islands,Male,Very good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,Yes,6.0,None of them,False,...,1.78,102.06,32.28,Yes,No,No,No,"Yes, received tetanus shot but not sure what type",No,No
246018,Virgin Islands,Female,Fair,0.0,7.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,False,...,1.93,90.72,24.34,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
246019,Virgin Islands,Male,Good,0.0,15.0,Within past year (anytime less than 12 months ...,Yes,7.0,1 to 5,False,...,1.68,83.91,29.86,Yes,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Yes
246020,Virgin Islands,Female,Excellent,2.0,2.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,False,...,1.70,83.01,28.66,No,Yes,Yes,No,"Yes, received tetanus shot but not sure what type",No,No


In [3]:
# One Hot Encoding 
heart_df = pd.get_dummies(heart_df, drop_first=True)
heart_df

,PhysicalHealthDays,MentalHealthDays,SleepHours,HadHeartAttack,HeightInMeters,WeightInKilograms,BMI,State_Alaska,State_Arizona,State_Arkansas,...,AlcoholDrinkers_Yes,HIVTesting_Yes,FluVaxLast12_Yes,PneumoVaxEver_Yes,"TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HighRiskLastYear_Yes,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
0,4.0,0.0,9.0,False,1.60,71.67,27.99,False,False,False,...,False,False,True,True,True,False,False,False,False,False
1,0.0,0.0,6.0,False,1.78,95.25,30.13,False,False,False,...,False,False,True,True,False,True,False,False,False,False
2,0.0,0.0,8.0,False,1.85,108.86,31.66,False,False,False,...,True,False,False,True,False,False,False,False,False,True
3,5.0,0.0,9.0,False,1.70,90.72,31.32,False,False,False,...,False,False,True,True,False,False,False,False,False,True
4,3.0,15.0,5.0,False,1.55,79.38,33.07,False,False,False,...,False,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,0.0,0.0,6.0,False,1.78,102.06,32.28,False,False,False,...,True,False,False,False,False,True,False,False,False,False
246018,0.0,7.0,7.0,False,1.93,90.72,24.34,False,False,False,...,False,False,False,False,False,False,False,False,False,True
246019,0.0,15.0,7.0,False,1.68,83.91,29.86,False,False,False,...,True,True,True,True,False,True,False,False,False,True
246020,2.0,2.0,7.0,False,1.70,83.01,28.66,False,False,False,...,False,True,True,False,False,True,False,False,False,False


In [4]:
def train_and_evaluate(X, y):
    # Modèle à utiliser
    model = XGBClassifier()
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Entraînement avec Cross Validation
    # Effectuer la validation croisée avec plusieurs métriques
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)

    # Enregistrez les résultats en prenant la moyenne des scores
    results = {
        "accuracy": cv_results['test_accuracy'].mean(),
        "precision": cv_results['test_precision_macro'].mean(),
        "recall": cv_results['test_recall_macro'].mean(),
        "f1": cv_results['test_f1_macro'].mean()
    }

    return results

In [5]:
X = heart_df.drop('HadHeartAttack', axis=1)
y = heart_df['HadHeartAttack']

results = train_and_evaluate(X, y)
results

{'accuracy': 0.9480696785085609,
 'precision': 0.7566405966833175,
 'recall': 0.6171010336745357,
 'f1': 0.6567297452180738}